# RAPORT 1 - KLASYFIKACJA 
<h4> Adrian Goik s23970 28.05.2024 - Indywidualny Program Studiów</h4>

<h3>TREŚĆ ZADANIA: Zadanie 3</h3>

W załączonym zbiorze danych <b><i>diabetes.csv </b></i>znajdują się dane kobiet indiańskiego pochodzenia z USA, które zachorowały lub nie zachorowały na cukrzycę. Klasyfikator ma na celu diagnozowanie choroby na podstawie parametrów medycznych kobiety. Sprawdź jak działają poznane klasyfikatory na tej bazie danych.<br> Dokonaj porównania:

<ol>
<li><b><i>k-NN, k=3</b></i></li>
<li><b><i>k-NN, k=5</b></i></li>
<li><b><i>k-NN, k=11</b></i></li>
<li><b><i>Naiwny bayesowski.</b></i></li>
<li><b><i>Drzewa decyzyjne.</b></i></li>
<ol>

# <p style="color:red">DO USUNIĘCIA!!</p>
Na wyższą liczbę punktów mają wpływ następujące rzeczy:</br> 
<ol>
<li>Dobra struktura sprawozdania (wprowadzenie, eksperymenty, objaśnienia, interpretacje).</li>
<li>Szczegółowość i dokładność eksperymentów, dodawanie komentarzy i własnych interpretacji wyników</li>
<li>Dobre przygotowanie bazy danych do eksperymentów</li>
<li>Przejrzystość i estetyka prezentacji</li>
<li>Nadprogramowe analizy mile widziane…</li>
</ol>

<h3> Wprowadzenie do wykonywanego zadania</h3>

tutaj zrobić wprowadzenie, uzupełniać w trakcie wykonywania i aktualizować o anaalizy

DODAJ: opis bazy danych i jakoś ją ogarnij -- jak to zrobić google ogarnaij sam


<h3> Opracowanie i przygotowanie bazy danych do eksperymentów</h3>

Na początek sprawdzę w jaki sposób rekordy bazy danych są wyświetlane i w jaki sposób trzeba przygotować bazę danych.

OPIS BAZY DANYCH wg. https://www.kaggle.com/datasets/mathchi/diabetes-data-set
<b><i>About this file</b></i>
<ol>
<li><b>Pregnancies:</b> Number of times pregnant</li>
<li><b>Glucose:</b> Plasma glucose concentration a 2 hours in an oral glucose tolerance test</li>
<li><b>BloodPressure:</b> Diastolic blood pressure (mm Hg)</li>
<li><b>SkinThickness:</b> Triceps skin fold thickness (mm)</li>
<li><b>Insulin:</b> 2-Hour serum insulin (mu U/ml)</li>
<li><b>BMI:</b> Body mass index (weight in kg/(height in m)^2)</li>
<li><b>DiabetesPedigreeFunction:</b> Diabetes pedigree function</li>
<li><b>Age:</b> Age (years)</li>
<li><b>Outcome:</b> Class variable (0 or 1)</li>
<ol>


<h5><p style="color:red">Użytkownik Elizabeth Camilleri zwróciła uwagę na ważny aspekt reprezentowanych danych. W niektórych rekordach dane nie są kompletne. Pojawia się np. skin thickness = 0. Jest to fizycznie niemożliwe. Spróbuję usunąć nielogiczne rekordy.</h5> </p>

<img src="Comment_db.png">

In [1]:
import pandas as pd

diabetes_db=pd.read_csv('diabetes.csv')
print(diabetes_db)



     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

### Sprawdzenie rekordów, które mają 0 w nielogicznych kategoriach: Glucose, BloodPressure, SkinThckness, BMI, Również Age wiemy, że badane miały co najmniej 21 lat
Pregnancies i Insulin może wynosić 0. Insulina może wynosić 0 właśnie w przypadku cukrzycy!!!

In [9]:
import pandas as pd

diabetes_db=pd.read_csv('diabetes.csv')

filtered_db = diabetes_db[(diabetes_db['Glucose'] != 0) & 
                 (diabetes_db['BloodPressure'] != 0) & 
                 (diabetes_db['SkinThickness'] != 0) & 
                 (diabetes_db['BMI'] != 0) & 
                 (diabetes_db['Age'] != 0)]

# Zapisz przefiltrowany DataFrame
filtered_db.to_csv('filtered_diabetes.csv', index=False)

print(filtered_db)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
6              3       78             50             32       88  31.0   
..           ...      ...            ...            ...      ...   ...   
761            9      170             74             31        0  44.0   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

Przefiltrowana baza pokazuje około 236 rekordów mniej. 

<b><i>po przygotowaniu bazy danych można przystąpić do treści zadania</b></i>

## Klasyfikator K-NN

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report



diabetes_db=pd.read_csv('filtered_diabetes.csv')

# Prepare the data for classification
X = diabetes_db.drop('Outcome', axis=1)  # Features
y = diabetes_db['Outcome']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Function to train and evaluate k-NN classifier
def evaluate_knn(k):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report

# Train and evaluate k-NN classifiers with k=3, k=5, k=11
results = {}
for k in [3, 5, 11]:
    accuracy, report = evaluate_knn(k)
    results[k] = {'accuracy': accuracy, 'report': report}

# Print the results
for k, result in results.items():
    print(f"k-NN with k={k}:")
    print(f"Accuracy: {result['accuracy']}")
    print("Classification Report:")
    print(result['report'])
    print("\n")

k-NN with k=3:
Accuracy: 0.7329545454545454
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.85      0.81       121
           1       0.59      0.47      0.53        55

    accuracy                           0.73       176
   macro avg       0.69      0.66      0.67       176
weighted avg       0.72      0.73      0.72       176



k-NN with k=5:
Accuracy: 0.7556818181818182
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       121
           1       0.64      0.51      0.57        55

    accuracy                           0.76       176
   macro avg       0.72      0.69      0.70       176
weighted avg       0.75      0.76      0.75       176



k-NN with k=11:
Accuracy: 0.7215909090909091
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.88      0.81       121
           1       0.58      0.

## Klasyfikator naiwny bayesowski

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

diabetes_db=pd.read_csv('filtered_diabetes.csv')

# Prepare the data for classification
X = diabetes_db.drop('Outcome', axis=1)  # Features
y = diabetes_db['Outcome']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.7757009345794392
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.80      0.83        74
           1       0.62      0.73      0.67        33

    accuracy                           0.78       107
   macro avg       0.74      0.76      0.75       107
weighted avg       0.79      0.78      0.78       107



## Drzewa Decyzyjne


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

diabetes_db=pd.read_csv('filtered_diabetes.csv')
# Prepare the data for classification
X = diabetes_db.drop('Outcome', axis=1)  # Features
y = diabetes_db['Outcome']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the Decision Tree classifier
tree_classifier = DecisionTreeClassifier(random_state=42)
tree_classifier.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = tree_classifier.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred)  # Store accuracy in the variable 'accuracy_tree'
report = classification_report(y_test, y_pred)

# Print the results in the specified format
print(f"Accuracy:  {accuracy_tree:.10f}")  # Format the accuracy to match the example
print("Classification Report:")
print(report)


Accuracy:  0.7102803738
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.76      0.78        74
           1       0.53      0.61      0.56        33

    accuracy                           0.71       107
   macro avg       0.67      0.68      0.67       107
weighted avg       0.72      0.71      0.72       107



## Naniesienie dokładności klasyfikatorów na wykres słupkowy

In [ ]:
data = []
data.append(accuracy_knn_k3_mean)
data.append(accuracy_knn_k5_mean)
data.append(accuracy_knn_k11_mean)
data.append(accuracy_bayes_mean)
data.append(accuracy_tree_mean)

categories = ['knn k=3', 'knn k=5', 'knn k=11', 'naive bayes', 'decision tree']

plt.bar(categories, data)

plt.xlabel('Category')
plt.ylabel('Values')  

plt.show()